In [1]:
import spconv.pytorch as spconv
import torch
from torch import nn

In [2]:
torch.manual_seed(2024)

In [3]:
spconv3d = spconv.SparseConv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [4]:
spconv3d.weight.shape

torch.Size([16, 3, 3, 3, 5])

In [5]:
conv3d = nn.Conv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [6]:
conv3d.weight.shape

torch.Size([16, 5, 3, 3, 3])

In [7]:
spconv3d_weights = torch.randn(16, 3, 3, 3, 5)

In [8]:
reshaped_weights = spconv3d_weights.permute(0, 4, 1, 2, 3)

In [9]:
conv3d.weight.data = reshaped_weights

In [10]:
spconv3d.weight.data = spconv3d_weights

In [11]:
torch.allclose(conv3d.weight.data.permute(0, 2, 3, 4, 1), spconv3d.weight.data)

True

In [12]:
# Parameters
N = 10  # number of non-zero points
num_channels = 5  # number of feature channels per point
ndim = 3  # number of spatial dimensions (e.g., 3D convolution)
batch_size = 2  # number of batches

# Generating random features
features = torch.randn(N, num_channels).to('cuda')

# Generating random indices for a 3D sparse tensor with batch indices
spatial_shape = (100, 100, 100)  # Define the spatial shape of the sparse tensor
indices = torch.zeros(N, ndim + 1, dtype=torch.int32).to('cuda')
indices[:, 0] = torch.randint(0, batch_size, (N,))  # Batch indices
for i in range(1, ndim + 1):
    indices[:, i] = torch.randint(0, spatial_shape[i - 1], (N,))  # Spatial indices

# Creating the SparseConvTensor
x = spconv.SparseConvTensor(features, indices, spatial_shape, batch_size)

# Converting sparse tensor to dense NCHW tensor (just for visualization; might be memory intensive)
x_dense_NCHW = x.dense()
print(x_dense_NCHW.shape)  # Should match the expected dense shape [batch_size, num_channels, *spatial_shape]


torch.Size([2, 5, 100, 100, 100])


In [13]:
x_dense_NCHW.unique()

tensor([-1.7558, -1.6769, -1.2789, -1.2409, -1.2103, -1.1418, -1.1062, -1.0178,
        -0.8062, -0.7696, -0.7256, -0.6933, -0.6913, -0.6305, -0.6193, -0.6103,
        -0.5825, -0.3765, -0.3565, -0.2757, -0.2683, -0.2475, -0.2196, -0.2154,
        -0.1974, -0.1909, -0.1677, -0.1347, -0.1217,  0.0000,  0.0141,  0.0477,
         0.0582,  0.1546,  0.2123,  0.3035,  0.3505,  0.4133,  0.4409,  0.5329,
         0.5396,  0.7577,  0.7669,  0.8155,  0.8365,  0.9001,  0.9055,  0.9997,
         1.1403,  1.5114,  1.6280], device='cuda:0')

In [14]:
x.features.unique()

tensor([-1.7558, -1.6769, -1.2789, -1.2409, -1.2103, -1.1418, -1.1062, -1.0178,
        -0.8062, -0.7696, -0.7256, -0.6933, -0.6913, -0.6305, -0.6193, -0.6103,
        -0.5825, -0.3765, -0.3565, -0.2757, -0.2683, -0.2475, -0.2196, -0.2154,
        -0.1974, -0.1909, -0.1677, -0.1347, -0.1217,  0.0141,  0.0477,  0.0582,
         0.1546,  0.2123,  0.3035,  0.3505,  0.4133,  0.4409,  0.5329,  0.5396,
         0.7577,  0.7669,  0.8155,  0.8365,  0.9001,  0.9055,  0.9997,  1.1403,
         1.5114,  1.6280], device='cuda:0')

In [15]:
conv3d.to('cuda')

Conv3d(5, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)

In [16]:
spconv3d.to('cuda')

SparseConv3d(5, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[0, 0, 0], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)

In [17]:
spconv_y = spconv3d(x)
y = conv3d(x_dense_NCHW)

In [18]:
spconv_y = spconv_y.dense()

In [19]:
spconv_y.unique()

tensor([-7.6593, -6.8356, -6.5969,  ...,  7.8996,  8.0378,  9.9962],
       device='cuda:0', grad_fn=<Unique2Backward0>)

In [23]:
y.unique()

tensor([-7.6599, -6.8354, -6.5960,  ...,  7.9015,  8.0381,  9.9956],
       device='cuda:0', grad_fn=<Unique2Backward0>)

In [20]:
l1loss = nn.L1Loss()

In [21]:
l1loss(spconv_y, y)

tensor(4.9212e-08, device='cuda:0', grad_fn=<MeanBackward0>)

In [22]:
torch.allclose(spconv_y, y)

False